In [1]:
from IPython.core.display import display, HTML
from nltk.corpus import stopwords, words
from nltk.stem import PorterStemmer
import numpy as np
import os
import pandas as pd
from string import punctuation
from textblob import Word

In [2]:
transcripts = pd.read_csv(os.path.join("data", "aamc_research_transcripts.csv"), index_col="interview_id", encoding="utf8")

In [3]:
q = transcripts.copy().iloc[0, :].to_frame()

In [4]:
contractions = { 
    "ain't": "am not / are not / is not / has not / have not",
    "aren't": "are not / am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is / how does",
    "i'd": "I had / I would",
    "i'd've": "I would have",
    "i'll": "I shall / I will",
    "i'll've": "I shall have / I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
}

In [5]:
def highlight(text, transformer=None, font_size=24, show_highlights=True):
    assert type(text) == str
    assert type(font_size) == int
    assert type(show_highlights) == bool
    
    if transformer == None:
        if show_highlights == True:
            display(HTML(text))
            return text
    
    highlights = text.split()
    
    for index, word in enumerate(highlights):
        if type(transformer) == list or type(transformer) == str:
            if word in transformer:
                highlights[index] = highlights[index].replace(word, f"""
                    <span style='color:red;font-size:{font_size}px;font-weight:bold;line-height:1px;'>{word}</span>""")
        elif type(transformer) == dict:
            if word in transformer:
                highlights[index] = highlights[index].replace(word, f"""
                    <span style='color:red;font-size:{font_size}px;font-weight:bold;line-height:1px;text-decoration:line-through;'>{word}</span>
                    <span style='color:blue;font-size:{font_size}px;font-weight:bold;line-height:1px;'>{transformer[word]}</span>""")
        else:
            if word != transformer(word):
                highlights[index] = highlights[index].replace(word, f"""
                    <span style='color:red;font-size:{font_size}px;font-weight:bold;line-height:1px;text-decoration:line-through;'>{word}</span>
                    <span style='color:blue;font-size:{font_size}px;font-weight:bold;line-height:1px;'>{transformer(word)}</span>""")
    
    highlights = " ".join(highlights)
    
    if show_highlights == True:
        display(HTML(highlights))
    
    return highlights

## Preprocessed question text
This question text is the original (preprocessed) question text from HireVue's interview transcripts.

In [6]:
for index, value in enumerate(q.index.tolist()):
    display(HTML(f"<strong>{value.capitalize()}</strong"))
    highlight(q.iloc[index, 0])
    print(" ")
;

''

## Process question text

### 1. Lowercase
Lowercase all the words in the question text.

In [7]:
for index, value in enumerate(q.index.tolist()):
    display(HTML(f"<strong>{value.capitalize()}</strong"))
    highlight(q.iloc[index, 0], lambda x: x.lower())
    print(" ")
;

''

In [8]:
q = q.apply(lambda x: x.str.lower())

### 2. Expand contractions
Expand all contractions in the question text (e.g., don't --> do not, I'm --> I am, etc.)

In [9]:
for index, value in enumerate(q.index.tolist()):
    display(HTML(f"<strong>{value.capitalize()}</strong"))
    highlight(q.iloc[index, 0], contractions)
    print(" ")
;

''

### 3. Remove punctuation
Remove all punctuation in the question text (e.g., commas, periods, etc.)

In [10]:
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [11]:
for index, value in enumerate(q.index.tolist()):
    display(HTML(f"<strong>{value.capitalize()}</strong"))
    highlight(q.iloc[index, 0], punctuation)
    print(" ")
;

''

In [12]:
q = q.apply(lambda x: x.str.replace("[{}]".format(punctuation), ""))

### 4. Remove stopwords
Remove all common frequently occurring stopwords in the question text (e.g., a, my, will, etc.)

In [13]:
sw = sorted(stopwords.words("english"))

In [14]:
print(", ".join(sw))

a, about, above, after, again, against, ain, all, am, an, and, any, are, aren, aren't, as, at, be, because, been, before, being, below, between, both, but, by, can, couldn, couldn't, d, did, didn, didn't, do, does, doesn, doesn't, doing, don, don't, down, during, each, few, for, from, further, had, hadn, hadn't, has, hasn, hasn't, have, haven, haven't, having, he, her, here, hers, herself, him, himself, his, how, i, if, in, into, is, isn, isn't, it, it's, its, itself, just, ll, m, ma, me, mightn, mightn't, more, most, mustn, mustn't, my, myself, needn, needn't, no, nor, not, now, o, of, off, on, once, only, or, other, our, ours, ourselves, out, over, own, re, s, same, shan, shan't, she, she's, should, should've, shouldn, shouldn't, so, some, such, t, than, that, that'll, the, their, theirs, them, themselves, then, there, these, they, this, those, through, to, too, under, until, up, ve, very, was, wasn, wasn't, we, were, weren, weren't, what, when, where, which, while, who, whom, why, w

In [15]:
for index, value in enumerate(q.index.tolist()):
    display(HTML(f"<strong>{value.capitalize()}</strong"))
    highlight(q.iloc[index, 0], sw)
    print(" ")
;

''

In [16]:
for index, _ in q.iterrows():
    q.loc[index] = q.loc[index].apply(lambda x: " ".join([word for word in x.split() if word not in sw]))

### 5. Singularize
Remove all plural words in the question text and replace them with singular words (e.g., friends --> friend ,pros --> pro, etc.)

In [17]:
for index, value in enumerate(q.index.tolist()):
    display(HTML(f"<strong>{value.capitalize()}</strong"))
    highlight(q.iloc[index, 0], lambda x: Word(x).singularize())
    print(" ")
;

''

In [18]:
for index, _ in q.iterrows():
    q.loc[index] = q.loc[index].apply(lambda x: " ".join([Word(word).singularize() for word in x.split()]))

### 6. Stem words
Remove all stems from words in the question text (e.g., applying --> appli, mentoring --> mentor, etc.)

In [19]:
stemmer = PorterStemmer()

In [20]:
for index, value in enumerate(q.index.tolist()):
    display(HTML(f"<strong>{value.capitalize()}</strong"))
    highlight(q.iloc[index, 0], stemmer.stem)
    print(" ")
;

''

In [21]:
for index, _ in q.iterrows():
    q.loc[index] = q.loc[index].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))

## Processed question text
This question text is the processed question text after applying each preprocessing step.

In [22]:
for index, value in enumerate(q.index.tolist()):
    display(HTML(f"<strong>{value.capitalize()}</strong"))
    highlight(q.iloc[index, 0])
    print(" ")
;

''